# Analítica en nube

Como vimos en clase, han sido unos años movidos en el mundo de la analítica pero a día de hoy contamos con sistemas nube que disponen de capacidades muy superiores a los modelos tradicionales de base de datos que conocíamos. El poder gestionar datos no tabulares o en formatos abiertos (CSV, PARQUET, JSON) y poder imponer capacidades de procesado más allá de SQL hacen que realmente casi cualquier proceso pueda ser integrado en los sistemas informacionales actuales.

## BigQuery

BigQuery es la oferta de Google a esa necesidad. Siendo los que iniciaron esta revolución cuentan con una de las plataformas más maduras en el sector. Todo usuario con cuenta Gmail puede acceder a la versión [Sandbox](https://cloud.google.com/bigquery/docs/sandbox) de BigQuery para poder probar su suerte.

Empezaremos por acceder a la consola de BigQuery:
https://console.cloud.google.com/bigquery 

![consola](img/consola-bq.png)

A priori no contáis con datos pero podréis añadir fuentes públicas directamente accesibles bajo consulta:

![data](img/add-data.png)

De hecho existe un compendio de datos directamente accesibles desde BigQuery. Por ejemplo, los datos de las bicicletas de Austin TX.

```sql
SELECT
    *
FROM
    `bigquery-public-data.austin_bikeshare.bikeshare_stations`
```

Por lo que podemos crear una plantilla para consultas y ejecutar nuestra consulta anterior directamente.

![query](img/query.png)

Podemos buscar datos de fechas concretas (año 2018).

```sql
SELECT
    *
FROM
    `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE
    EXTRACT(YEAR FROM start_time) = 2018
LIMIT 10
```

O combinarlo con información de las estaciones para poder obtener gráficos sobre tiempos y duraciones en base a los días de la semana.

```sql
SELECT
    start_station_name,
    EXTRACT(HOUR FROM start_time) AS start_hour,
    EXTRACT(DAYOFWEEK FROM start_time) AS day_of_week,
    duration_minutes
FROM
    `bigquery-public-data.austin_bikeshare.bikeshare_trips` AS trips
JOIN
    `bigquery-public-data.austin_bikeshare.bikeshare_stations` AS stations
ON
    trips.start_station_name = stations.name
WHERE
    EXTRACT(YEAR FROM start_time) = 2018
    AND duration_minutes > 0
```

![query-plot](img/query-plot.png)

## Snowflake

Existen alternativas multinube de cara a no ligarse a un proveedor de por vida. Snowflake es sin duda el líder en este ámbito en lo que a sistemas RDBMS destinados a analítica se refiere. Su versión de prueba nos permite crear un entorno válido por un mes e interactuar con este como si de una base de datos convencional se tratara.

[Crear cuenta de prueba](https://signup.snowflake.com/)

Una vez dispongáis de los datos para conectaros solo tendréis que configurar el acceso programático. Snowflake dispone de un conector propio que debería facilitarnos la tareas. Además es compatible con [Pandas](https://quickstarts.snowflake.com/guide/getting_started_with_snowpark_pandas/index.html#0).

In [13]:
# !pip install "snowflake-connector-python[pandas]"

In [1]:
import snowflake.connector

con = snowflake.connector.connect(
    user='<user_name>',
    password='<password>',
    account='<account>',
    database="SNOWFLAKE_SAMPLE_DATA"
)

In [2]:
import pandas as pd

customers = pd.read_sql_query("SELECT * FROM TPCH_SF10.CUSTOMER", con=con)
customers.head(4)

/tmp/ipykernel_10982/957583243.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers = pd.read_sql_query("SELECT * FROM TPCH_SF10.CUSTOMER", con=con)


,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,285001,Customer#000285001,4FeDDwg7xsqSVT,15,25-854-361-6981,3057.88,MACHINERY,"ve the regular, pending packages affix caref"
1,285002,Customer#000285002,UbQAgu44NZOvMadwuhc8IKs1fYIMz5ZNmlRYzyl,11,21-625-459-4267,-190.59,MACHINERY,ress deposits. furiously even packages sleep c...
2,285003,Customer#000285003,"Mug,CjxX0lpHalGRJeG3RCGEdCJFZXAYQ6IvlW4",12,22-998-713-3526,6636.14,AUTOMOBILE,"ideas. regular, special platelets nag furiously"
3,285004,Customer#000285004,rQ4SNwFDmtrNKgzwlxQ219j5Qef A,3,13-245-434-3165,2607.20,BUILDING,nal ideas. requests across the regular deposit...


Tenéis más información sobre el conjunto de datos [aquí](https://docs.snowflake.com/en/user-guide/sample-data-tpch). Y diréis ¿por qué es importante saber usar SQL? Bueno, si nos pudiéramos traer todo a Pandas...

```python
orders = pd.read_sql_query("SELECT * FROM TPCH_SF1000.ORDERS", con=con)
orders.head(4)
```

## Pregunta 1

Para los elementos en la lista de pedidos, información relativa a aquellos cuyo día de envío sea 90 días previo a la fecha dada (01 de Diciembre de 1998).

In [3]:
query = """
select
	l_returnflag,
	l_linestatus,
	sum(l_quantity) as sum_qty,
	sum(l_extendedprice) as sum_base_price,
	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
	avg(l_quantity) as avg_qty,
	avg(l_extendedprice) as avg_price,
	avg(l_discount) as avg_disc,
	count(*) as count_order
from
	TPCH_SF1.lineitem
where
	l_shipdate <= DATEADD(day, -90, '1998-12-01')
group by
	l_returnflag,
	l_linestatus
order by
	l_returnflag,
	l_linestatus;
"""

result = pd.read_sql_query(query, con=con)
result

/tmp/ipykernel_10982/2564960835.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, con=con)


,L_RETURNFLAG,L_LINESTATUS,SUM_QTY,SUM_BASE_PRICE,SUM_DISC_PRICE,SUM_CHARGE,AVG_QTY,AVG_PRICE,AVG_DISC,COUNT_ORDER
0,A,F,37734107.0,5.658655e+10,5.375826e+10,5.590907e+10,25.522006,38273.129735,0.049985,1478493
1,N,F,991417.0,1.487505e+09,1.413082e+09,1.469649e+09,25.516472,38284.467761,0.050093,38854
2,N,O,74476040.0,1.117017e+11,1.061182e+11,1.103670e+11,25.502227,38249.117989,0.049997,2920374
3,R,F,37719753.0,5.656804e+10,5.374129e+10,5.588962e+10,25.505794,38250.854626,0.050009,1478870


### Pregunta 2

Encuentra el proveedor en regiones concretas que puede proveed cada parte de un pedido a su mínimo precio. 

In [4]:
query = """
select
	s_acctbal,
	s_name,
	n_name,
	p_partkey,
	p_mfgr,
	s_address,
	s_phone,
	s_comment
from
	TPCH_SF1.part,
	TPCH_SF1.supplier,
	TPCH_SF1.partsupp,
	TPCH_SF1.nation,
	TPCH_SF1.region
where
	p_partkey = ps_partkey
	and s_suppkey = ps_suppkey
	and p_size = 15
	and p_type like '%BRASS'
	and s_nationkey = n_nationkey
	and n_regionkey = r_regionkey
	and r_name = 'EUROPE'
	and ps_supplycost = (
		select
			min(ps_supplycost)
		from
			TPCH_SF1.partsupp,
			TPCH_SF1.supplier,
			TPCH_SF1.nation,
			TPCH_SF1.region
		where
			p_partkey = ps_partkey
			and s_suppkey = ps_suppkey
			and s_nationkey = n_nationkey
			and n_regionkey = r_regionkey
			and r_name = 'EUROPE'
	)
order by
	s_acctbal desc,
	n_name,
	s_name,
	p_partkey LIMIT 100;
"""

result = pd.read_sql_query(query, con=con)
result

/tmp/ipykernel_10982/1463180599.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(query, con=con)


,S_ACCTBAL,S_NAME,N_NAME,P_PARTKEY,P_MFGR,S_ADDRESS,S_PHONE,S_COMMENT
0,9938.53,Supplier#000005359,UNITED KINGDOM,185358,Manufacturer#4,"QKuHYh,vZGiwu2FWEJoLDx04",33-429-790-6131,uriously regular requests hag
1,9937.84,Supplier#000005969,ROMANIA,108438,Manufacturer#1,"ANDENSOSmk,miq23Xfb5RWt6dvUcvt6Qa",29-520-692-3537,efully express instructions. regular requests ...
2,9936.22,Supplier#000005250,UNITED KINGDOM,249,Manufacturer#4,B3rqp0xbSEim4Mpy2RH J,33-320-228-2957,etect about the furiously final accounts. slyl...
3,9923.77,Supplier#000002324,GERMANY,29821,Manufacturer#4,y3OD9UywSTOk,17-779-299-1839,ackages boost blithely. blithely regular depos...
4,9871.22,Supplier#000006373,GERMANY,43868,Manufacturer#5,J8fcXWsTqM,17-813-485-8637,etect blithely bold asymptotes. fluffily ironi...
...,...,...,...,...,...,...,...,...
95,7887.08,Supplier#000009792,GERMANY,164759,Manufacturer#3,Y28ITVeYriT3kIGdV2K8fSZ V2UqT5H1Otz,17-988-938-4296,ckly around the carefully fluffy theodolites. ...
96,7871.50,Supplier#000007206,RUSSIA,104695,Manufacturer#1,3w fNCnrVmvJjE95sgWZzvW,32-432-452-7731,ironic requests. furiously final theodolites c...
97,7852.45,Supplier#000005864,RUSSIA,8363,Manufacturer#4,"WCNfBPZeSXh3h,c",32-454-883-3821,usly unusual pinto beans. brave ideas sleep ca...
98,7850.66,Supplier#000001518,UNITED KINGDOM,86501,Manufacturer#1,ONda3YJiHKJOC,33-730-383-3892,ifts haggle fluffily pending pai


Tenéis más ejemplos de preguntas con cierto carácter analítico en el siguiente [repositorio](https://github.com/SatoriCyber/snowflake-tpch-timing/blob/master/tpch.sql). 

# Abstracciones DataFrames

Existen soluciones que nos permiten interactuar empleando las APIs conocidas (Pandas) pero impactando directamente sobre la base de datos con queries. Hay que prestar atención porque las consultas no se lanzan hasta que se requiere pintar los resultados (`.show()`, `.collect()`, `.to_pandas()`)

In [6]:
# !pip install snowflake-snowpark-python

In [7]:
from snowflake.snowpark import Session

connection_parameters = {
    "account": "<your snowflake account>",
    "user": "<your snowflake user>",
    "password": "<your snowflake password>",
    "database" : "SNOWFLAKE_SAMPLE_DATA"
}  

new_session = Session.builder.configs(connection_parameters).create()  

Una vez iniciada la sesión podemos vincular variables locales a tablas en la base de datos.

In [12]:
df_region = new_session.table("TPCH_SF1.region")

Unicamente registra el nexo entre la variable y la tabla, de manera que si quisiéramos obtener esos datos dispondríamos de la consulta que los muestra.

In [13]:
df_region.queries

{'queries': ['SELECT  *  FROM (TPCH_SF1.region)'], 'post_actions': []}

`.show` lanza la consulta y nos muestra los 10 elementos iniciales de la tabla.

In [15]:
df_region.show(10)

------------------------------------------------------------------------------------
|"R_REGIONKEY"  |"R_NAME"     |"R_COMMENT"                                         |
------------------------------------------------------------------------------------
|0              |AFRICA       |lar deposits. blithely final packages cajole. r...  |
|1              |AMERICA      |hs use ironic, even requests. s                     |
|2              |ASIA         |ges. thinly even pinto beans ca                     |
|3              |EUROPE       |ly final courts cajole furiously final excuse       |
|4              |MIDDLE EAST  |uickly special accounts cajole carefully blithe...  |
------------------------------------------------------------------------------------



Mientras que `.to_pandas`retorna los datos a un objeto pandas en nuestra máquina.

In [16]:
df_region.to_pandas()

,R_REGIONKEY,R_NAME,R_COMMENT
0,0,AFRICA,lar deposits. blithely final packages cajole. ...
1,1,AMERICA,"hs use ironic, even requests. s"
2,2,ASIA,ges. thinly even pinto beans ca
3,3,EUROPE,ly final courts cajole furiously final excuse
4,4,MIDDLE EAST,uickly special accounts cajole carefully blith...


Podemos generar abstracciones mayores, por ejemplo tablas que sean joins de varias.

In [17]:
df_nation = new_session.table("TPCH_SF1.nation")

In [19]:
df_nation_region = df_nation.join(df_region, "R_REGIONKEY" == "N_REGIONKEY")
df_nation_region.queries

{'queries': ['SELECT  *  FROM (( SELECT "N_NATIONKEY" AS "N_NATIONKEY", "N_NAME" AS "N_NAME", "N_REGIONKEY" AS "N_REGIONKEY", "N_COMMENT" AS "N_COMMENT" FROM TPCH_SF1.nation) AS SNOWPARK_LEFT INNER JOIN ( SELECT "R_REGIONKEY" AS "R_REGIONKEY", "R_NAME" AS "R_NAME", "R_COMMENT" AS "R_COMMENT" FROM TPCH_SF1.region) AS SNOWPARK_RIGHT)'],
 'post_actions': []}

Una vez requerimos una acción es cuando se lanza la consulta construida.

In [20]:
df_nation_region.show(10)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"N_NATIONKEY"  |"N_NAME"   |"N_REGIONKEY"  |"N_COMMENT"                                         |"R_REGIONKEY"  |"R_NAME"     |"R_COMMENT"                                         |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0              |ALGERIA    |0              | haggle. carefully final deposits detect slyly ...  |0              |AFRICA       |lar deposits. blithely final packages cajole. r...  |
|0              |ALGERIA    |0              | haggle. carefully final deposits detect slyly ...  |1              |AMERICA      |hs use ironic, even requests. s                     |
|0              |ALGERIA    |0              | haggle. carefully final deposits detect slyl

Podemos anidar filtros y agregaciones y únicamente al incluir la acción final es cuando se lanzarán las consultas.

In [27]:
from snowflake.snowpark.functions import col

df_filtrado_algeria = df_nation_region.filter(col("N_NAME") == "ALGERIA")

In [28]:
df_filtrado_algeria

In [29]:
df_filtrado_algeria.queries

{'queries': ['SELECT  *  FROM ( SELECT  *  FROM (( SELECT "N_NATIONKEY" AS "N_NATIONKEY", "N_NAME" AS "N_NAME", "N_REGIONKEY" AS "N_REGIONKEY", "N_COMMENT" AS "N_COMMENT" FROM TPCH_SF1.nation) AS SNOWPARK_LEFT INNER JOIN ( SELECT "R_REGIONKEY" AS "R_REGIONKEY", "R_NAME" AS "R_NAME", "R_COMMENT" AS "R_COMMENT" FROM TPCH_SF1.region) AS SNOWPARK_RIGHT)) WHERE ("N_NAME" = \'ALGERIA\')'],
 'post_actions': []}

In [30]:
df_filtrado_algeria.to_pandas()

,N_NATIONKEY,N_NAME,N_REGIONKEY,N_COMMENT,R_REGIONKEY,R_NAME,R_COMMENT
0,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,0,AFRICA,lar deposits. blithely final packages cajole. ...
1,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,1,AMERICA,"hs use ironic, even requests. s"
2,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,2,ASIA,ges. thinly even pinto beans ca
3,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,3,EUROPE,ly final courts cajole furiously final excuse
4,0,ALGERIA,0,haggle. carefully final deposits detect slyly...,4,MIDDLE EAST,uickly special accounts cajole carefully blith...


In [24]:
df_nation_region.group_by("N_NAME").count()

In [25]:
df_nation_region.group_by("N_NAME").count().queries

{'queries': ['SELECT "N_NAME", count(1) AS "COUNT" FROM ( SELECT  *  FROM (( SELECT "N_NATIONKEY" AS "N_NATIONKEY", "N_NAME" AS "N_NAME", "N_REGIONKEY" AS "N_REGIONKEY", "N_COMMENT" AS "N_COMMENT" FROM TPCH_SF1.nation) AS SNOWPARK_LEFT INNER JOIN ( SELECT "R_REGIONKEY" AS "R_REGIONKEY", "R_NAME" AS "R_NAME", "R_COMMENT" AS "R_COMMENT" FROM TPCH_SF1.region) AS SNOWPARK_RIGHT)) GROUP BY "N_NAME"'],
 'post_actions': []}

In [26]:
df_nation_region.group_by("N_NAME").count().show()

------------------------
|"N_NAME"    |"COUNT"  |
------------------------
|ALGERIA     |5        |
|ARGENTINA   |5        |
|BRAZIL      |5        |
|CANADA      |5        |
|ETHIOPIA    |5        |
|GERMANY     |5        |
|INDIA       |5        |
|IRAN        |5        |
|IRAQ        |5        |
|MOZAMBIQUE  |5        |
------------------------



Al finalizar nuestras acciones cerraremos la conexión.

In [31]:
new_session.close()

Tenéis más información en la documentación oficial de Snowflake:

* [Documentación](https://docs.snowflake.com/en/user-guide-getting-started)
* [Snowpark API](https://docs.snowflake.com/en/developer-guide/snowpark/python/index)

Existe un proyecto más generalista pensado para todo tipo de bases de datos (no solo Snowflake) llamado [Apache Ibis](https://ibis-project.org/). Son abstracciones importantes cuando estamos familiarizados con el mundo de los dataframes pero queremos actuar sobre bases de datos voluminosas.